In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from time import sleep
import os
import pandas as pd

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
url = 'https://www.udemy.com/courses/IT-and-software/'
page = requests.get(url).text
soup = BeautifulSoup(page, 'lxml')
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(url)

In [ ]:
def extract_record(item):
    
    title = item.find('div',{'class':'udlite-focus-visible-target udlite-heading-md course-card--course-title--vVEjC'}).text
    description = item.find('p',{'class':'udlite-text-sm course-card--course-headline--2DAqq'}).text
    price = item.find('div', {'class':'price-text--price-part--2npPm course-card--discount-price--1bQ5Q udlite-heading-md'}).text 
    rating = item.find('span',{'class':'star-rating--star-wrapper--1QyBg'}).find('span').text
    reviews = item.find('span',{'class':'udlite-text-xs course-card--reviews-text--1yloi'}).text
    discount = item.find('s').find('span').text
    trainer = item.find('div',{'class':'udlite-text-xs course-card--instructor-list--nH1OC'}).text
    course_info = item.find('div',{'class':'udlite-text-xs course-card--row--29Y0w course-card--course-meta-info--2jTzN'}).find_all('span')
    
    if len(course_info) == 3 :
        total_hours = course_info[0].text
        total_lectures = course_info[1].text
        level = course_info[2].text
    else :
        total_hours = None
        total_lectures = None
        level = course_info[1].text
    
    result = (title,description,price,rating,reviews,discount,trainer,total_hours,total_lectures,level) 
    
    
    return result

In [ ]:
def get_data() :
    records=[]

    for i in range(1,620):
        url = 'https://www.udemy.com/courses/it-and-software/?p='+str(i)
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(driver.page_source, "lxml")

        div = soup.find('div',class_='course-list--container--3zXPS')
        results = div.find_all('a',{'class':'udlite-custom-focus-visible browse-course-card--link--3KIkQ'})

        for item in results:
            records.append(extract_record(item))
    
    df = pd.DataFrame(records, columns=['title','description','price','rating','reviews',
                                    'discount','trainer','total_hours','total_lectures',
                                    'level'])
    return df
    

In [ ]:
#df = get_data()

In [ ]:
#df.to_csv('udemy_data.csv')

In [2]:
df = pd.read_csv('udemy_data.csv')

In [3]:
df.head()

,Unnamed: 0,title,description,price,rating,reviews,discount,trainer,total_hours,total_lectures,level
0,0,The Complete Ethical Hacking Course: Beginner ...,"Learn how to do ethical hacking, penetration t...",Current price$17.99,Rating: 4.1 out of 5,"(28,341)",$99.99,"Ermin Kreponic, Aldin Omerdic",25 total hours,113 lectures,All Levels
1,1,AZ-303 Azure Architecture Technologies Exam Pr...,Prove your Azure Architect Technology skills t...,Current price$14.99,Rating: 4.5 out of 5,"(25,690)",$69.99,"Scott Duffy • 550.000+ Students, Software Arc...",12.5 total hours,152 lectures,Intermediate
2,2,AZ-104 Microsoft Azure Administrator Exam Cert...,Prove your AZ-104 Microsoft Azure Administrato...,Current price$14.99,Rating: 4.6 out of 5,"(25,341)",$39.99,"Scott Duffy • 550.000+ Students, Software Arc...",12.5 total hours,162 lectures,Intermediate
3,3,CompTIA Security+ (SY0-601) Complete Course & ...,CompTIA Security+ (SY0-601) Bootcamp - Your pr...,Current price$17.99,Rating: 4.7 out of 5,"(24,998)",$99.99,"Jason Dion • 600,000+ Enrollments Worldwide, D...",21.5 total hours,274 lectures,All Levels
4,4,CCNA 2020 200-125 Video Boot Camp With Chris B...,"Join The 90,000+ Students Who Are Learning Rea...",Current price$14.99,Rating: 4.7 out of 5,"(24,753)",$19.99,Chris Bryant,62.5 total hours,467 lectures,Beginner


In [6]:
df.columns

Index(['Unnamed: 0', 'title', 'description', 'price', 'rating', 'reviews',
       'discount', 'trainer', 'total_hours', 'total_lectures', 'level'],
      dtype='object')

In [8]:
df.drop(columns="Unnamed: 0",inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9904 entries, 0 to 9903
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           9904 non-null   object
 1   description     9904 non-null   object
 2   price           9904 non-null   object
 3   rating          9904 non-null   object
 4   reviews         9904 non-null   object
 5   discount        9904 non-null   object
 6   trainer         9904 non-null   object
 7   total_hours     8666 non-null   object
 8   total_lectures  8666 non-null   object
 9   level           9904 non-null   object
dtypes: object(10)
memory usage: 773.9+ KB


In [12]:
df.dropna(inplace=True)

In [13]:
df.trainer = df.trainer.str.replace(',','')

In [14]:
df.trainer = df.trainer.str.split().str[0] +' '+ df.trainer.str.split().str[1]

In [15]:
df.head()

,title,description,price,rating,reviews,discount,trainer,total_hours,total_lectures,level
0,The Complete Ethical Hacking Course: Beginner ...,"Learn how to do ethical hacking, penetration t...",Current price$17.99,Rating: 4.1 out of 5,"(28,341)",$99.99,Ermin Kreponic,25 total hours,113 lectures,All Levels
1,AZ-303 Azure Architecture Technologies Exam Pr...,Prove your Azure Architect Technology skills t...,Current price$14.99,Rating: 4.5 out of 5,"(25,690)",$69.99,Scott Duffy,12.5 total hours,152 lectures,Intermediate
2,AZ-104 Microsoft Azure Administrator Exam Cert...,Prove your AZ-104 Microsoft Azure Administrato...,Current price$14.99,Rating: 4.6 out of 5,"(25,341)",$39.99,Scott Duffy,12.5 total hours,162 lectures,Intermediate
3,CompTIA Security+ (SY0-601) Complete Course & ...,CompTIA Security+ (SY0-601) Bootcamp - Your pr...,Current price$17.99,Rating: 4.7 out of 5,"(24,998)",$99.99,Jason Dion,21.5 total hours,274 lectures,All Levels
4,CCNA 2020 200-125 Video Boot Camp With Chris B...,"Join The 90,000+ Students Who Are Learning Rea...",Current price$14.99,Rating: 4.7 out of 5,"(24,753)",$19.99,Chris Bryant,62.5 total hours,467 lectures,Beginner


In [16]:
import re
def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return "".join(num)
def find_rating(text):
    num = re.findall(r'[0-9]+',text)
    num.pop()
    return ".".join(num)

In [17]:
df.reviews = df['reviews'].apply(lambda x: find_number(x)).astype(int)

In [19]:
df.rating = df['rating'].apply(lambda x: find_rating(x)).astype(float)

In [33]:
df.total_hours = df['total_hours'].str.extract(r'(\d+.\d*)').astype('float')

In [38]:
df.discount = df['discount'].str.extract(r'(\d+.\d*)').astype('float')

In [43]:
df.total_lectures = df['total_lectures'].apply(lambda x: find_number(x)).astype(int)

In [49]:
df.price = df['price'].str.extract(r'(\d+.\d*)').astype('float')

In [46]:
df.level.unique()

array(['All Levels', 'Intermediate', 'Beginner'], dtype=object)

In [50]:
df

,title,description,price,rating,reviews,discount,trainer,total_hours,total_lectures,level
0,The Complete Ethical Hacking Course: Beginner ...,"Learn how to do ethical hacking, penetration t...",17.99,4.1,28341,99.99,Ermin Kreponic,25.0,113,All Levels
1,AZ-303 Azure Architecture Technologies Exam Pr...,Prove your Azure Architect Technology skills t...,14.99,4.5,25690,69.99,Scott Duffy,12.5,152,Intermediate
2,AZ-104 Microsoft Azure Administrator Exam Cert...,Prove your AZ-104 Microsoft Azure Administrato...,14.99,4.6,25341,39.99,Scott Duffy,12.5,162,Intermediate
3,CompTIA Security+ (SY0-601) Complete Course & ...,CompTIA Security+ (SY0-601) Bootcamp - Your pr...,17.99,4.7,24998,99.99,Jason Dion,21.5,274,All Levels
4,CCNA 2020 200-125 Video Boot Camp With Chris B...,"Join The 90,000+ Students Who Are Learning Rea...",14.99,4.7,24753,19.99,Chris Bryant,62.5,467,Beginner
...,...,...,...,...,...,...,...,...,...,...
9898,The Absolute Beginners Guide to Cyber Security...,"Learn Cyber Security concepts such as hacking,...",21.99,4.6,20207,99.99,Alexander Oni,4.0,58,Beginner
9899,Cisco CCNA 200-301 Exam: Complete Course with ...,The complete course to help you prepare and pa...,14.99,4.7,19873,19.99,David Bombal,79.0,687,Beginner
9901,Spring Framework Master Class - Java Spring th...,Learn the magic of Java Spring Framework with ...,21.99,4.4,19715,129.99,in28Minutes Official,12.0,138,All Levels
9902,Amazon Web Services (AWS) Certified - 4 Certif...,"Videos, labs & practice exams - AWS Certified ...",14.99,4.5,19203,89.99,BackSpace Academy,44.5,194,All Levels
